In [ ]:
import simpy
import random
import matplotlib.pyplot as plt

# Constants
VEHICLE_ARRIVAL_RATE_A = 30  # seconds (Street A)
VEHICLE_ARRIVAL_RATE_B = 30 # seconds (Street B)
RED_LIGHT_DURATION_A = 15  # seconds
TIME_TO_PASS_THROUGH = 2  # seconds
SIM_TIME = 3600  # simulation time in seconds (60 minute)

# Functions
def vehicle(env, resource, wait_times, street_name, arrival_time, light_status, vehicle_number):
    # arrival_time = round(env.now, 0)
    light_state = yield light_status.get()
    print(f"Vehicle {vehicle_number}-{street_name} arrived at time {arrival_time} and light status was {light_state}.")
    # with resource.request() as request:
    #     yield request
    #     while True:
    #         light_state = yield light_status.get()
    #         if (street_name == "Street A" and light_state == 'green_a') or (street_name == "Street B" and light_state == 'green_b'):
    #             break
    #         # yield light_status.put(light_state)  # Put the light state back if it's not green for the current street
    #         yield env.timeout(1)  # Wait until the light turns green
    #     wait_time = env.now - arrival_time
    #     wait_times.append(wait_time)
    #     print(f"Vehicle {vehicle_number}-{street_name} passed at {env.now} because status was {light_state} with wait time {wait_time}.")
    #     yield env.timeout(TIME_TO_PASS_THROUGH)  # time to pass through
    while True:
        if (street_name == "Street A" and light_state == 'green_a') or (street_name == "Street B" and light_state == 'green_b'):
            break
        # yield light_status.put(light_state)  # Put the light state back if it's not green for the current street
        yield env.timeout(1)  # Wait until the light turns green
        light_state = yield light_status.get()
    wait_time = round(env.now,0) - arrival_time
    wait_times.append(wait_time)
    print(f"Vehicle {vehicle_number}-{street_name} passed at {env.now} because status was {light_state} with wait time {wait_time}.")
    yield env.timeout(TIME_TO_PASS_THROUGH)  # time to pass through

def vehicle_generator(env, resource, wait_times, arrival_rate, street_name, light_status):
    counter = 0
    while True:
        yield env.timeout(random.expovariate(1 / arrival_rate))
        counter += 1
        env.process(vehicle(env, resource, wait_times, street_name, light_status, counter))

def traffic_light(env, light_status, green_light_duration_a, red_light_duration_a):
    green_light_duration_b = red_light_duration_a

    # while True:
    #     # Green light for Street A, Red light for Street B
    #     yield light_status.put('green_a')
    #     yield env.timeout(green_light_duration_a)
        
    #     yield light_status.put('red')
    #     yield env.timeout(red_light_duration_a)
        
    #     # Green light for Street B, Red light for Street A
    #     yield light_status.put('green_b')
    #     yield env.timeout(green_light_duration_b)
        
    #     yield light_status.put('red')
    #     yield env.timeout(red_light_duration_a)

    while True:
        # Green light for Street A, Red light for Street B
        print(f"Time {env.now}: GREEN light ON for Street A, RED light ON for Street B")
        yield light_status.put('green_a')
        yield env.timeout(green_light_duration_a)
        
        print(f"Time {env.now}: GREEN light ON for Street B, RED light ON for Street A")
        yield light_status.put('green_b')
        yield env.timeout(red_light_duration_a)
    
def simulate_intersection(env, resource, wait_times_a, wait_times_b, green_light_duration_a, red_light_duration_a):
    light_status = simpy.Store(env)
    env.process(vehicle_generator(env, resource, wait_times_a, VEHICLE_ARRIVAL_RATE_A, "Street A", light_status))
    env.process(vehicle_generator(env, resource, wait_times_b, VEHICLE_ARRIVAL_RATE_B, "Street B", light_status))
    env.process(traffic_light(env, light_status, green_light_duration_a, red_light_duration_a))
    env.run(until=SIM_TIME)

# Simulation and plotting
green_light_durations = range(15, 30, 2)  # Different green light durations
average_wait_times_a = []
average_wait_times_b = []

for green_light_duration_a in green_light_durations:
    wait_times_a = []
    wait_times_b = []
    env = simpy.Environment()
    traffic_light_resource = simpy.Resource(env, capacity=1)
    simulate_intersection(env, traffic_light_resource, wait_times_a, wait_times_b, green_light_duration_a, RED_LIGHT_DURATION_A)
    
    avg_wait_time_a = sum(wait_times_a) / len(wait_times_a) if wait_times_a else 0
    avg_wait_time_b = sum(wait_times_b) / len(wait_times_b) if wait_times_b else 0
    
    average_wait_times_a.append(avg_wait_time_a)
    average_wait_times_b.append(avg_wait_time_b)

# Plotting the results
plt.plot(green_light_durations, average_wait_times_a, label='Avg Wait Time Street A')
plt.plot(green_light_durations, average_wait_times_b, label='Avg Wait Time Street B')
plt.xlabel('Green Light A Duration (seconds)')
plt.ylabel('Average Wait Time (seconds)')
plt.title('Average Wait Time vs. Green Light Duration')
plt.legend()
plt.grid(True)
plt.show()


In [1]:
import simpy
import random
import matplotlib.pyplot as plt

# Constants
VEHICLE_ARRIVAL_RATE_A = 30  # seconds (Street A)
VEHICLE_ARRIVAL_RATE_B = 30 # seconds (Street B)
RED_LIGHT_DURATION_A = 15  # seconds
TIME_TO_PASS_THROUGH = 2  # seconds
SIM_TIME = 3600  # simulation time in seconds (60 minute)

# Functions
def vehicle(env, resource, wait_times, street_name, light_status, vehicle_number):
    arrival_time = round(env.now, 0)
    light_state = yield light_status.get()
    print(f"Vehicle {vehicle_number}-{street_name} arrived at time {arrival_time} and light status was {light_state}.")

    while True:
        if (street_name == "Street A" and light_state == 'green_a') or (street_name == "Street B" and light_state == 'green_b'):
            break
        # yield light_status.put(light_state)  # Put the light state back if it's not green for the current street
        yield env.timeout(1)  # Wait until the light turns green
        light_state = yield light_status.get()

    wait_time = round(env.now,0) - arrival_time
    wait_times.append(wait_time)
    print(f"Vehicle {vehicle_number}-{street_name} passed at {env.now} because status was {light_state} with wait time {wait_time}.")

    yield env.timeout(TIME_TO_PASS_THROUGH)  # time to pass through

def vehicle_generator(env, resource, wait_times, arrival_rate, street_name, light_status):
    counter = 0
    while True:
        yield env.timeout(random.expovariate(1 / arrival_rate))
        counter += 1
        env.process(vehicle(env, resource, wait_times, street_name, light_status, counter))

def traffic_light(env, light_status, green_light_duration_a, red_light_duration_a):

    while True:
        # Green light for Street A, Red light for Street B
        print(f"Time {env.now}: GREEN light ON for Street A, RED light ON for Street B")
        yield light_status.put('green_a')
        yield env.timeout(green_light_duration_a)
        
        print(f"Time {env.now}: GREEN light ON for Street B, RED light ON for Street A")
        yield light_status.put('green_b')
        yield env.timeout(red_light_duration_a)
    
def simulate_intersection(env, resource, wait_times_a, wait_times_b, green_light_duration_a, red_light_duration_a):
    light_status = simpy.Store(env)
    env.process(vehicle_generator(env, resource, wait_times_a, VEHICLE_ARRIVAL_RATE_A, "Street A", light_status))
    env.process(vehicle_generator(env, resource, wait_times_b, VEHICLE_ARRIVAL_RATE_B, "Street B", light_status))
    env.process(traffic_light(env, light_status, green_light_duration_a, red_light_duration_a))
    env.run(until=SIM_TIME)

# Simulation and plotting
green_light_durations = range(15, 30, 2)  # Different green light durations
average_wait_times_a = []
average_wait_times_b = []

for green_light_duration_a in green_light_durations:
    wait_times_a = []
    wait_times_b = []
    env = simpy.Environment()
    traffic_light_resource = simpy.Resource(env, capacity=1)
    simulate_intersection(env, traffic_light_resource, wait_times_a, wait_times_b, green_light_duration_a, RED_LIGHT_DURATION_A)
    
    avg_wait_time_a = sum(wait_times_a) / len(wait_times_a) if wait_times_a else 0
    avg_wait_time_b = sum(wait_times_b) / len(wait_times_b) if wait_times_b else 0
    
    average_wait_times_a.append(avg_wait_time_a)
    average_wait_times_b.append(avg_wait_time_b)

# Plotting the results
plt.plot(green_light_durations, average_wait_times_a, label='Avg Wait Time Street A')
plt.plot(green_light_durations, average_wait_times_b, label='Avg Wait Time Street B')
plt.xlabel('Green Light A Duration (seconds)')
plt.ylabel('Average Wait Time (seconds)')
plt.title('Average Wait Time vs. Green Light Duration')
plt.legend()
plt.grid(True)
plt.show()


Time 0: GREEN light ON for Street A, RED light ON for Street B


TypeError: vehicle() missing 1 required positional argument: 'vehicle_number'